In [121]:
import psycopg2
import psycopg2.extras as extras

import pandas as pd
from dotenv import dotenv_values

from datetime import datetime,date
from dateutil import relativedelta
import os

# Paramter

In [122]:
is_only_admin=0
is_only_admin=bool(is_only_admin)



# Get the next month 
* Set Window Sheduler to monthly run at the last day of The month
* No matter what day you run this job , the program will  get PMs of the  next month

In [123]:

#dt = datetime.strptime(datetime(2023,12,4,10,0).strftime('%Y-%m-%d'),'%Y-%m-%d')
dtNow=datetime.now()
dt = datetime.strptime(dtNow.strftime('%Y-%m-%d'),'%Y-%m-%d')
print(dt)

first_day_next_month = dt + relativedelta.relativedelta(months=1, day=1)
first_day_2next_month = dt + relativedelta.relativedelta(months=2, day=1)
print(first_day_next_month )   
print(first_day_2next_month ) 


2023-11-29 00:00:00
2023-12-01 00:00:00
2024-01-01 00:00:00


# Create File Name and Path to attach to email

In [124]:
file_name=f"PM_{first_day_next_month.strftime('%b%y')}_{dtNow.strftime('%d%m%y%H%M')}.xlsx"
if is_only_admin==False:
   file_name=f"SM-{file_name}" 
else:
   file_name=f"None-SM-{file_name}" 
file_name

'SM-PM_Dec23_2911230611.xlsx'

In [125]:
env_path=r'D:\PythonDev\Yit\SMartApp_ScriptDev\.env'
config = dotenv_values(dotenv_path=env_path)

In [126]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

# Retrive data

In [127]:
# https://github.com/technqvi/SMartApp/blob/main/app/pm_doc_manager/pm_export.py

sql_pm=f"""
    select ac.company_full_name as "ชื่อลูกค้า",
    ap.contract_no as "เลขที่สัญญา",ap.enq_id as "ENQ" ,
    ap.project_name as "ชื่อโครงการ",
    TO_CHAR(pm.planned_date,'Mon YYYY') as "แผนจะทำPM",
    TO_CHAR(pm.ended_pm_date,'DD Mon YYYY') as "วันสุดท้ายที่ทำPM",
    pm.remark as  "งวดPM",
    ae.employee_name as "หัวหน้าทีม",
    (select emp.employee_name emp from app_employee emp where emp.id=pm.engineer_id ) as "Engineer"
    from app_preventivemaintenance pm
    left join app_project ap on ap.id = pm.project_id
    left join app_company ac on ac.id = ap.company_id
    left join app_employee ae on ae.id =pm.team_lead_id
    where  pm.planned_date>='{first_day_next_month}' and pm.planned_date<'{first_day_2next_month}'
    and ac.is_managed_by_admin = {is_only_admin}
    order by  ac.company_full_name,ap.enq_id,pm.remark
    """
# print(sql_pm)

In [128]:
dfPM=list_data(sql_pm,None,get_postgres_conn())
dfPM.info()
dfPM.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ชื่อลูกค้า         114 non-null    object
 1   เลขที่สัญญา        0 non-null      object
 2   ENQ                114 non-null    object
 3   ชื่อโครงการ        114 non-null    object
 4   แผนจะทำPM          114 non-null    object
 5   วันสุดท้ายที่ทำPM  114 non-null    object
 6   งวดPM              114 non-null    object
 7   หัวหน้าทีม         114 non-null    object
 8   Engineer           99 non-null     object
dtypes: object(9)
memory usage: 8.1+ KB


,ชื่อลูกค้า,เลขที่สัญญา,ENQ,ชื่อโครงการ,แผนจะทำPM,วันสุดท้ายที่ทำPM,งวดPM,หัวหน้าทีม,Engineer
0,"Bangkok Airways Public Co.,Ltd.",None,ENQ/65-454,MA for Bangkok Airway (SAP HANA MANGO 2023),Dec 2023,31 Dec 2023,4/4 - Cisco Network,Thitipong Limudomsuk,Attasit somsuk
1,"Bangkok Airways Public Co.,Ltd.",None,ENQ/65-454,MA for Bangkok Airway (SAP HANA MANGO 2023),Dec 2023,31 Dec 2023,4/4 - Cisco UCS Chassis,Yutthana Klinmala,Tura Klinmala
2,"Bangkok Airways Public Co.,Ltd.",None,ENQ/65-454,MA for Bangkok Airway (SAP HANA MANGO 2023),Dec 2023,31 Dec 2023,4/4 - EMC VNX,Boonlert Kawta,Akkachit Krasasung
3,"Bangkok Airways Public Co.,Ltd.",None,ENQ/65-454,MA for Bangkok Airway (SAP HANA MANGO 2023),Dec 2023,31 Dec 2023,4/4 - HPE,Boonlert Kawta,Akkachit Krasasung
4,"Bangkok Airways Public Co.,Ltd.",None,ENQ/65-454,MA for Bangkok Airway (SAP HANA MANGO 2023),Dec 2023,31 Dec 2023,4/4 - Trend Micro Cloud One,Chaipus Aupatamwipanon,Aod Wongswat


In [129]:
sql_item = f"""
 select  ac.company_full_name as "ชื่อลูกค้า",
 ap.contract_no as "เลขที่สัญญา",ap.enq_id as "ENQ" , ap.project_name as "ชื่อโครงการ",

             ai.serial_number as "Serial",
             (select  productype_name from app_product_type where id=ai.product_type_id ) as "ProudctType",
             (select  brand_name from app_brand where id=ai.brand_id ) as "Brand",
              (select  model_name from app_model where id=ai.model_id ) as "Model",

             TO_CHAR(pm.planned_date,'Mon YYYY') as "แผนจะทำPM",TO_CHAR(pm.ended_pm_date,'DD Mon YYYY') as "วันสุดท้ายที่ทำPM",
             pm.remark as  "งวดPM",ae.employee_name as "หัวหน้าทีม",
             (select emp.employee_name emp from app_employee emp where emp.id=pm.engineer_id ) as "Planed Engineer",

           (select employee_name from app_employee eng_pm  where eng_pm.id=pm_item.pm_engineer_id ) as "Operation Engineer",
           TO_CHAR(pm_item.actual_date,'DD Mon YYYY') as "ActualDate",pm_item.call_number as "Call Number",

            (select employee_name from app_employee eng_doc  where eng_doc.id=pm_item.document_engineer_id ) as "Doc Engineer",
           TO_CHAR(pm_item.document_date,'DD Mon YYYY') as "DocumentDate",pm_item.pm_document_number as "Doc Number",
           pm_item.remark as "Remark"

from app_pm_inventory as pm_item
left join app_inventory ai on ai.id = pm_item.inventory_id
-- inner join app_product_type  product_type on ai.product_type_id = product_type.id
left join app_preventivemaintenance pm on pm.id = pm_item.pm_master_id
left join app_project ap on ap.id = pm.project_id
left join app_company ac on ac.id = ap.company_id
left join app_employee ae on ae.id =pm.team_lead_id

where pm_item.is_pm=True and ac.is_managed_by_admin = {is_only_admin}
and  pm.planned_date>='{first_day_next_month}' and pm.planned_date<'{first_day_2next_month}'

order by  ac.company_full_name,ap.enq_id,pm.remark

    """

# print(sql_item)

In [130]:
dfItem=list_data(sql_item,None,get_postgres_conn())
dfItem.info()
dfItem.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 821 entries, 0 to 820
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ชื่อลูกค้า          821 non-null    object
 1   เลขที่สัญญา         0 non-null      object
 2   ENQ                 821 non-null    object
 3   ชื่อโครงการ         821 non-null    object
 4   Serial              821 non-null    object
 5   ProudctType         821 non-null    object
 6   Brand               821 non-null    object
 7   Model               821 non-null    object
 8   แผนจะทำPM           821 non-null    object
 9   วันสุดท้ายที่ทำPM   821 non-null    object
 10  งวดPM               821 non-null    object
 11  หัวหน้าทีม          821 non-null    object
 12  Planed Engineer     707 non-null    object
 13  Operation Engineer  56 non-null     object
 14  ActualDate          56 non-null     object
 15  Call Number         0 non-null      object
 16  Doc Engineer        56 non

,ชื่อลูกค้า,เลขที่สัญญา,ENQ,ชื่อโครงการ,Serial,ProudctType,Brand,Model,แผนจะทำPM,วันสุดท้ายที่ทำPM,งวดPM,หัวหน้าทีม,Planed Engineer,Operation Engineer,ActualDate,Call Number,Doc Engineer,DocumentDate,Doc Number,Remark
0,"Bangkok Airways Public Co.,Ltd.",None,ENQ/65-454,MA for Bangkok Airway (SAP HANA MANGO 2023),FOX1714GD5X,Switch,Cisco,Nexus N5K-C5596UP-FA,Dec 2023,31 Dec 2023,4/4 - Cisco Network,Thitipong Limudomsuk,Attasit somsuk,None,None,None,None,None,None,None
1,"Bangkok Airways Public Co.,Ltd.",None,ENQ/65-454,MA for Bangkok Airway (SAP HANA MANGO 2023),FGL172010LM,Hardware,Cisco,CISCO2911/K9,Dec 2023,31 Dec 2023,4/4 - Cisco Network,Thitipong Limudomsuk,Attasit somsuk,None,None,None,None,None,None,None
2,"Bangkok Airways Public Co.,Ltd.",None,ENQ/65-454,MA for Bangkok Airway (SAP HANA MANGO 2023),FOX1705H1AW,Switch,Cisco,Nexus N5K-C5596UP-FA,Dec 2023,31 Dec 2023,4/4 - Cisco Network,Thitipong Limudomsuk,Attasit somsuk,None,None,None,None,None,None,None
3,"Bangkok Airways Public Co.,Ltd.",None,ENQ/65-454,MA for Bangkok Airway (SAP HANA MANGO 2023),SSI170505SF,Switch,Cisco,CON-OSP-2232PFA,Dec 2023,31 Dec 2023,4/4 - Cisco Network,Thitipong Limudomsuk,Attasit somsuk,None,None,None,None,None,None,None
4,"Bangkok Airways Public Co.,Ltd.",None,ENQ/65-454,MA for Bangkok Airway (SAP HANA MANGO 2023),SSI170804RK,Switch,Cisco,CON-OSP-2232PFA,Dec 2023,31 Dec 2023,4/4 - Cisco Network,Thitipong Limudomsuk,Attasit somsuk,None,None,None,None,None,None,None


# Excel file

In [132]:
with pd.ExcelWriter(file_name) as writer:
    dfPM.to_excel(writer, sheet_name="PM-Plan",index=False)
    dfItem.to_excel(writer, sheet_name="PM-Item",index=False)

# Email Office 365

In [ ]:
# Delete attach files

In [ ]:
os.remove(file_name)